In [1]:
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
from matplotlib.collections import PatchCollection
import numpy as np
import os
import re
from scipy import signal
from scipy import ndimage

%matplotlib

np.random.seed(77777)

Using matplotlib backend: Qt5Agg


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [2]:
def gen_template(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)
    ax.scatter([0, 0, w_px, w_px], [0, h_px, 0, h_px], color='k')

    # black background
    # ax.add_patch(Rectangle((0, 0), w_px, h_px, color='k'))
    pattern_size = 8
    pattern = np.random.random((pattern_size,pattern_size)) * 0.5 + .5
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='w')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='w')) # vert

    circ_diam = c_size
    if loc == 'SW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4, h_px / 4),
        )
    elif loc == 'NW':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'NE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, 3 * h_px / 4),
        )
    elif loc == 'SE':
        locs = (
            (circ_diam / 2, 0),
            (w_px / 4 + circ_diam / 2, 0),
            (2 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4 + circ_diam / 2, 0),
            (3 * w_px / 4, h_px / 4),
        )
    elif loc == 'raft':
        locs = (
            (w_px / 4, h_px / 4),
            (w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, 3 * h_px / 4),
            (3 * w_px / 4, h_px / 4),
        )
    else:
        raise ValueError(f'loc {loc} is not an option')

    for elem in locs:
        ax.add_patch(Circle(elem, circ_diam / 2, color='w', zorder=1))

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

plt.close('all')
locs = ['SW', 'NW', 'NE', 'SE', 'raft']
TARGET_W = 0.04 # 40 mm
TARGET_H = 0.04
# for loc in locs:
    # gen_template(TARGET_W, TARGET_H, 300, loc, plot=False)

In [3]:
def measure_pos(filename, plot_corr_metrics=False):
    locs = ['SW', 'NW', 'NE', 'SE', 'raft'] # all targets expected to be in the images

    # The main figure to host the ground truth image, xcorr, and template
    f0 = plt.figure(constrained_layout=True)
    f0.suptitle(filename)
    axd = f0.subplot_mosaic(
        """
        ac
        ab
        """
    )
    # Ground truth
    im = image.imread(filename)
    im = np.sum(im.astype(float), axis=2) # single channel
    im = np.max(im) - im # invert value
    im /= im.max() # normalize
    im -= im.mean() # detrend
    stride = 4 # downsample for speed
    # im = ndimage.rotate(im, -90)
    im = im[::stride,::stride]
    axd['a'].imshow(im, cmap='Greys')
    axd['b'].set_title('Similarity Metric (xcorr)')
    axd['c'].set_title(f'Target Template Image')

    f1, ax = plt.subplots(nrows=len(locs))
    f1.suptitle(filename)
    f1.tight_layout()

    # record the best correlation position in pixels and the template scale factor, pixels per meter
    found_pos = []
    px_per_ms = []
    for i, loc in enumerate(locs):
        ax[i].set_title(loc)
        ax[i].set_xlabel('Template Image Size (px)')
        ax[i].set_ylabel('Correlation\nPeak-to-Sidelobe Ratio')
        l = None # placeholder for matplotlib line object

        tmp = image.imread(loc + '_target.png')
        tmp = np.sum(tmp.astype(float) , axis=2) # single channel
        tmp = np.max(tmp) - tmp # invert
        tmp /= tmp.max() # normalize
        tmp -= tmp.mean() # detrend
        # try different scales to find the size of the template, in pixels.
        # Proper range for this will depend on the angular size of the target in the image.
        pxs = np.logspace(6.5, 8, num=30, base=2) / stride
        sz = np.max(tmp.shape)
        zooms = pxs / sz
        psrs = []
        for j, zoom in enumerate(zooms):
            # scale template image
            tmp_zoom = ndimage.zoom(tmp, zoom)
            axd['c'].imshow(tmp_zoom, cmap='Greys')
            # find best matching lags in image space
            xcorr = signal.fftconvolve(im, tmp_zoom[::-1, ::-1], mode='valid')
            axd['b'].imshow(xcorr)
            # a metric for strength of the match
            sidelobe = np.partition(xcorr.flatten(), -2)[-2] # https://stackoverflow.com/q/33181350
            psr = np.max(xcorr) / sidelobe
            psrs.append(psr)
            # print(f'xcorr peak-to-sidelobe ratio: {psr}')
            if not l:
                l, = ax[i].plot(pxs[:len(psrs)], psrs)
            else:
                l.set_data(pxs[:len(psrs)], psrs)
                ax[i].set_xlim(.9 * min(pxs[:len(psrs)]), max(pxs[:len(psrs)]) * 1.1)
                ax[i].set_ylim(.9 * min(psrs), max(psrs) * 1.1)
            
            f0.canvas.draw_idle()
            f1.canvas.draw_idle()
            plt.pause(0.001)

        # Generally, it seems a match has been found when the PSR reaches a global maximum.
        best_idx = np.argmax(psrs)
        # recover zoomed template
        tmp_zoom = ndimage.zoom(tmp, zooms[best_idx])
        # recover xcorr
        xcorr = signal.fftconvolve(im, tmp_zoom[::-1, ::-1], mode='valid')
        # the indices of the best match lags
        y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
        x_cent = x + tmp_zoom.shape[0] / 2
        y_cent = y + tmp_zoom.shape[1] / 2
        print(f'{loc} target center found at {(x_cent * stride, y_cent * stride)} px in image')
        axd['a'].add_patch(Rectangle((x, y), tmp_zoom.shape[0], tmp_zoom.shape[1], fill=False, color='red'))
        axd['a'].scatter(x_cent, y_cent)
        axd['a'].annotate(f'{loc}, est. size = {pxs[best_idx]:.2f} px', (x,y))
        px_per_ms.append(pxs[best_idx] * stride / (TARGET_W))
        found_pos.append((x_cent * stride, y_cent * stride))
        print(f'Calculated pixels per m: {pxs[best_idx] * stride / (TARGET_W)}')
                
        if loc == 'raft':
            # Go on to try to find the best rotation
            im_crop = im[y:y + tmp_zoom.shape[1], x:x + tmp_zoom.shape[0]]
            rot_psrs = []
            angs = np.arange(-10, 10, .1)
            for ang in angs:
                im_crop_rot = ndimage.rotate(im_crop, ang, reshape=False)
                rot_xcorr = signal.fftconvolve(im_crop_rot, tmp_zoom[::-1, ::-1], mode='valid')
                rot_sidelobe = np.partition(xcorr.flatten(), -2)[-2] # https://stackoverflow.com/q/33181350
                rot_psrs.append(np.max(rot_xcorr) / rot_sidelobe)
            best_rot = angs[np.argmax(rot_psrs)]
            print('Raft rotation:', best_rot)
            axd['c'].set_title(f'Raft target after derotation by {best_rot} deg:')
            axd['c'].imshow(ndimage.rotate(im_crop, best_rot, reshape=False))
            

    file_base = os.path.splitext(os.path.basename(filename))[0]
    f0.savefig(f'{file_base}_found_locs.png')
    f1.savefig(f'{file_base}_psrs.png')

    # post-processing: calculate actual distances
    avg_px_per_m = np.array(px_per_ms).mean()

    # which position is the raft in?
    sw_loc = np.array(found_pos[0]) / avg_px_per_m
    nw_loc = np.array(found_pos[1]) / avg_px_per_m
    ne_loc = np.array(found_pos[2]) / avg_px_per_m
    se_loc = np.array(found_pos[3]) / avg_px_per_m
    raft_loc = np.array(found_pos[-1]) / avg_px_per_m

    dist_x = raft_loc[0] - sw_loc[0]
    dist_y = sw_loc[1] - raft_loc[1]

    # What position is the raft expected to be in, based on the input filename?
    m = re.search(r'xp[0-9]+', fname)
    x_expect = float(m.group(0).lstrip('xp')) / 100
    m = re.search(r'yp[0-9]+', fname)
    y_expect = float(m.group(0).lstrip('yp')) / 100
    print('filename:', filename)
    print('expected pos:', (x_expect, y_expect))
    print(f'(x,y): {(dist_x, dist_y)}')
    print(f'Error: {np.array([x_expect, y_expect]) - np.array((dist_x, dist_y))}')

In [4]:
num_pts = 0
for x_elem in np.arange(.15, .5, .05):
    for y_elem in np.arange(.15, .45, .05):
        num_pts += 1
        fname = 'xp' + str(np.round(x_elem, 2)).lstrip('0.') + 'yp' + str(np.round(y_elem, 2)).lstrip('0.') + '.jpeg'
        print(f'Processing {fname}')
        if os.path.exists('./' + fname):
            measure_pos(fname)

Processing xp15yp15.jpeg
SW target center found at (1252.0, 3016.0) px in image
Calculated pixels per m: 4015.718420469423
NW target center found at (1246.0, 630.0) px in image
Calculated pixels per m: 3874.2954430775376
NE target center found at (2918.0, 610.0) px in image
Calculated pixels per m: 3874.2954430775376
SE target center found at (2952.0, 3000.0) px in image
Calculated pixels per m: 4015.718420469423


KeyboardInterrupt: 

In [15]:
def create_files_in_order(directory):
    '''
    Make copies of an rename files according to their position in the error map grid.
    Assumed that photo filenames are ordered such that they depict each position in
    increasing rows of x, increasing columns of y, row-major.
    '''
    from shutil import copyfile
    import glob

    orig_photos = sorted(glob.glob(os.path.join('img', 'orig', '*.JPG')))
    print(len(orig_photos))
    i = 0
    for y_elem in np.arange(.15, .55, .05):
        for x_elem in np.arange(.15, .55, .05):
            fname = 'xp' + str(np.round(x_elem, 2)).lstrip('0.') + 'yp' + str(np.round(y_elem, 2)).lstrip('0.') + '.jpeg'
            # print(x_elem, y_elem, orig_photos[i], os.path.join('img', fname))
            copyfile(orig_photos[i], os.path.join('img', fname))
            i += 1
            # print(i)

create_files_in_order('./')

64
